In [11]:
import tensorflow as tf
import numpy as np
from __future__ import print_function
from six.moves import cPickle as pickle
from six.moves import range
import time

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validaition set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validaition set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size*image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


#### training

In [16]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    
    #input data
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, num_labels])
    )
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    )
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases
    )    

#### computation & iteration

In [17]:
num_steps = 801

def accuracy(predictions, labels):
    return (100*(np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)))
        / predictions.shape[0])

tic = time.clock()
with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction])
    
        if(step % 100 == 0):
            print('Loss at step %d: %f' % (step, loss_value))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
                test_prediction.eval(), test_labels))
toc = time.clock()
toc-tic

initialized
Loss at step 0: 15.498641
Training accuracy: 8.0%
Validation accuracy: 11.0%
Loss at step 100: 2.327559
Training accuracy: 72.0%
Validation accuracy: 71.0%
Loss at step 200: 1.859489
Training accuracy: 75.0%
Validation accuracy: 73.0%
Loss at step 300: 1.608500
Training accuracy: 76.0%
Validation accuracy: 74.0%
Loss at step 400: 1.441538
Training accuracy: 77.0%
Validation accuracy: 74.0%
Loss at step 500: 1.317751
Training accuracy: 78.0%
Validation accuracy: 75.0%
Loss at step 600: 1.220653
Training accuracy: 78.0%
Validation accuracy: 75.0%
Loss at step 700: 1.141837
Training accuracy: 79.0%
Validation accuracy: 75.0%
Loss at step 800: 1.076436
Training accuracy: 79.0%
Validation accuracy: 75.0%
Test accuracy: 82.0%


110.03671500000002

#### SGD

In [28]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(
        tf.truncated_normal([image_size**2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [29]:
num_steps = 3001

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 19.215664
Miniibatch accruacy: 7.0%
Validation accuracy: 11.0%
Minibatch loss at step 500: 1.904208
Miniibatch accruacy: 68.0%
Validation accuracy: 75.0%
Minibatch loss at step 1000: 1.314883
Miniibatch accruacy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 1500: 1.553318
Miniibatch accruacy: 73.0%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 1.146971
Miniibatch accruacy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 1.112160
Miniibatch accruacy: 72.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000: 0.714170
Miniibatch accruacy: 83.0%
Validation accuracy: 78.0%
Test accuracy: 85.0%


10.217679999999973

#### SGD + ReLu

In [33]:
batch_size = 128
num_hidden = 1024

graph = tf.Graph()
with graph.as_default():
    
    # mini-batch => placeholder
    
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape = (batch_size, image_size**2))
    tf_train_labels = tf.placeholder(tf.float32,
                                    shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(
        tf.truncated_normal([image_size**2, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    weights_2 = tf.Variable(
         tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training
    logits_1 = tf.nn.relu(
        tf.matmul(tf_train_dataset, weights_1) + biases_1
    )
    logits_2 = tf.matmul(logits_1, weights_2) + biases_2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits_2, tf_train_labels
        )
    )
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits_2)
    valid_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )
    test_prediction = tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights_1) + biases_1)
            ,weights_2)
        + biases_2
    )

In [34]:
num_steps = 3001

tic = time.clock()

with tf.Session(graph = graph) as sess:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range (num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels : batch_labels}
        
        _, loss_value, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, loss_value))
            print('Miniibatch accruacy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

toc = time.clock()
toc-tic

initialized
Minibatch loss at step 0: 336.081604
Miniibatch accruacy: 17.0%
Validation accuracy: 37.0%
Minibatch loss at step 500: 11.417870
Miniibatch accruacy: 75.0%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 10.953521
Miniibatch accruacy: 75.0%
Validation accuracy: 76.0%
Minibatch loss at step 1500: 5.983901
Miniibatch accruacy: 77.0%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 3.116799
Miniibatch accruacy: 87.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 5.189484
Miniibatch accruacy: 82.0%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 3.278154
Miniibatch accruacy: 81.0%
Validation accuracy: 81.0%
Test accuracy: 88.0%


231.76974699999994